# Star Wars RAG Chatbot

## Project Purpose
This notebook implements a Retrieval-Augmented Generation (RAG) chatbot specialized in Star Wars knowledge. By combining the Star Wars API (SWAPI) with modern NLP techniques, we create an interactive question-answering system that:

- Retrieves accurate Star Wars information from a curated knowledge base
- Generates contextually relevant, lore-accurate responses
- Provides an engaging chat interface with a Jedi historian persona
- Demonstrates practical implementation of RAG architecture using industry-standard tools

**Key Technologies:**
- RAG Framework: LangChain with LCEL (LangChain Expression Language)
- Embeddings: SentenceTransformers (all-MiniLM-L6-v2)
- Vector Store: Chroma
- LLM: Google's Gemini
- UI: Gradio

In [38]:
#from google.colab import files
#files.upload()


{}

## 1. Install Dependencies

First, we need to install all the required Python packages.

In [ ]:
# Install all required packages for Star Wars RAG Chatbot
!pip install -q -U langchain langchain-google-genai sentence-transformers chromadb requests gradio


## 2. Configure API Keys

We need a Google API key to use the Gemini model. We'll use Colab's secrets manager to handle the key securely.

In [ ]:
#from dotenv import load_dotenv
#import os

#load_dotenv(".env")  # .env dosyasını oku
#os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [45]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


## Dataset Description

Our knowledge base is constructed from the Star Wars API (SWAPI), which provides canonical information across six main categories:

1. **Characters** (`people`):
   - Biographical data (name, birth year, physical characteristics)
   - Personal attributes (gender, species associations)

2. **Films**:
   - Movie details (title, episode number)
   - Production information (director, producer, release date)

3. **Extended Universe** (multiple endpoints):
   - Planets: Climate, terrain, population
   - Species: Classification, attributes, homeworld
   - Vehicles & Starships: Technical specifications, roles

**Data Processing:**
- Each API response is converted to natural language passages
- Structured fields are formatted for optimal retrieval
- Relations between entities are preserved
- Text chunks are sized for effective RAG retrieval

The processed dataset serves as the foundational knowledge for our chatbot's responses, ensuring accuracy and canonical consistency.

In [64]:
import requests
import json

def get_all_data(api_url):
    """Fetches all pages of data from a SWAPI endpoint."""
    results = []
    while api_url:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            results.extend(data['results'])
            api_url = data['next']
        else:
            print(f"Failed to fetch data from {api_url}. Status code: {response.status_code}")
            break
    return results

def process_data_to_text(data, category):
    """Convert SWAPI data to QA-friendly text passages."""
    text_passages = []
    for item in data:
        if category == "people":
            passage = f"Name: {item.get('name')}\n"
            passage += f"Height: {item.get('height')}\n"
            passage += f"Mass: {item.get('mass')}\n"
            passage += f"Hair color: {item.get('hair_color')}\n"
            passage += f"Skin color: {item.get('skin_color')}\n"
            passage += f"Eye color: {item.get('eye_color')}\n"
            passage += f"Birth year: {item.get('birth_year')}\n"
            passage += f"Gender: {item.get('gender')}\n"

        elif category == "films":
            passage = f"Film: {item.get('title')}\n"
            passage += f"Episode: {item.get('episode_id')}\n"
            passage += f"Director: {item.get('director')}\n"
            passage += f"Producer: {item.get('producer')}\n"
            passage += f"Release date: {item.get('release_date')}\n"
        else:  # planets, species, vehicles, starships
            passage = f"Category: {category}\n"
            for key, value in item.items():
              if isinstance(value, list):
               value = ", ".join([str(v) for v in value])
               passage += f"{key.replace('_', ' ').capitalize()}: {value}\n"


        text_passages.append(passage)
    return text_passages


base_url = "https://swapi.dev/api/"
categories = ["people", "planets", "starships", "vehicles", "species", "films"]
all_text_passages = []

for category in categories:
    print(f"Fetching data for {category}...")
    data = get_all_data(base_url + category + "/")
    text_passages = process_data_to_text(data, category)
    all_text_passages.extend(text_passages)
    print(f"Finished fetching and processing {len(data)} items for {category}.")

print(f"\nTotal text passages created: {len(all_text_passages)}")


Fetching data for people...
Finished fetching and processing 82 items for people.
Fetching data for planets...
Finished fetching and processing 60 items for planets.
Fetching data for starships...
Finished fetching and processing 36 items for starships.
Fetching data for vehicles...
Finished fetching and processing 39 items for vehicles.
Fetching data for species...
Finished fetching and processing 37 items for species.
Fetching data for films...
Finished fetching and processing 6 items for films.

Total text passages created: 260


## Methods: RAG Pipeline Implementation

Our RAG architecture consists of four main components:

1. **Text Embeddings (SentenceTransformers)**:
   - Model: `all-MiniLM-L6-v2`
   - Converts text passages into dense vector representations
   - Optimized for semantic similarity search

2. **Vector Store (Chroma)**:
   - In-memory vector database for development
   - Efficient similarity search capabilities
   - Metadata support for categorical filtering

3. **Retrieval Chain (LangChain)**:
   - Top-k retrieval (k=5) for relevant context
   - Custom prompt template with Jedi historian persona
   - Zero-shot question answering approach

4. **Generation (Gemini)**:
   - Uses Google's Gemini model via LangChain
   - Context-aware response generation
   - Maintains Star Wars universe authenticity

In [65]:
!pip install -q chromadb sentence-transformers

from chromadb.utils import embedding_functions
from chromadb import Client
from sentence_transformers import SentenceTransformer

# Modeli yükle
model = SentenceTransformer("all-MiniLM-L6-v2")

# Chroma client oluştur
client = Client()

# Koleksiyon oluştur
collection = client.create_collection("starwars", get_or_create=True)

# Text passage'leri ekle
for i, text in enumerate(all_text_passages):
    embedding = model.encode(text).tolist()  # embeddingi oluştur
    collection.add(
        ids=[str(i)],                  # burada id veriyoruz
        documents=[text],
        metadatas=[{"category": "starwars"}],
        embeddings=[embedding],
    )

print("Chroma collection created successfully!")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.4.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.33.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-sdk~=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is inc

## 5. Create the RAG Retrieval Chain

Now we'll build the RAG pipeline using LangChain Expression Language (LCEL). This will connect our retriever, a prompt template, and the Gemini model.

In [42]:
!pip install langchain==0.0.274 langchain-google-genai

  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires ope

In [93]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.prompt import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from chromadb import Client

# 1️⃣ Chroma client ve var olan koleksiyona bağlan
client = Client()
collection = client.get_collection("starwars")

# 2️⃣ Retriever oluştur
from langchain.vectorstores import Chroma
vector_db = Chroma(
    collection_name="starwars",
    client=client
)

# Bu, LLM’e en alakalı 5 passage vererek daha doğru cevap alınmasını sağlar.
retriever = vector_db.as_retriever(search_kwargs={"k": 5})


# 3️⃣ Generative modeli başlat
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

template = """
You are a wise Jedi historian and Star Wars lore expert, residing in the Jedi Archives on Coruscant.
Your duty is to answer questions from travelers across the galaxy using only the knowledge preserved in the Archives, provided in the context below.

You must never fabricate or invent information beyond what is written in the Archives.
If a detail is missing or uncertain, respond with: “It appears that the Archives hold no record of such knowledge.”

Provide your responses in a detailed, lore-accurate, and immersive manner, consistent with the tone of the Star Wars universe.

---
📜 Context (from the Archives):
{context}

🛰️ Question from the Traveler:
{question}

💫 Jedi Historian’s Answer:
"""



prompt = PromptTemplate(template=template, input_variables=["context", "question"])


# Helper function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 5️⃣ RAG zincirini oluştur (✅ Düzenlendi)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | RunnableLambda(lambda x: llm.invoke(x.to_string())["content"])  # sadece content al
)


print("RAG chain created successfully!")

RAG chain created successfully!


## Results and Interface

The implemented chatbot achieves several key objectives:

1. **Knowledge Integration**:
   - Successfully indexes all SWAPI endpoints
   - Maintains entity relationships in vector space
   - Provides factual, canonical responses

2. **Response Quality**:
   - Contextually relevant answers
   - Character-aware responses through Jedi historian persona
   - Graceful handling of unknown information

3. **User Interface (Gradio)**:
   - Clean, intuitive chat interface
   - Real-time response generation
   - Mobile-responsive design
   - Easy deployment to Hugging Face Spaces

**Example Usage**:
- Ask about character details, movie facts, or universe lore
- Get responses grounded in official Star Wars canon
- Explore relationships between characters, places, and events

The chatbot demonstrates effective combination of RAG architecture with modern LLM capabilities, providing an engaging way to explore Star Wars knowledge.

In [94]:

from langchain_google_genai import ChatGoogleGenerativeAI

# Modeli güncel Gemini modeli ile başlat
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Soru sor
answer = llm.invoke("What is Leia Organa’s birth year?")
print(answer)



content='Leia Organa was born in **19 BBY** (Before the Battle of Yavin). This is the same year as her twin brother, Luke Skywalker.'


In [96]:
import gradio as gr
import time

chat_history = []

# 🔹 RAG zincirini oluştur (Gradio’dan önce)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | RunnableLambda(lambda x: str(llm.invoke(x.to_string())))
)


def ask_question(question):
    answer = rag_chain.invoke(question)
    if isinstance(answer, dict) and "content" in answer:
        answer = answer["content"]

    chat_history.append({"role": "user", "content": question})
    chat_history.append({"role": "assistant", "content": answer})

    return chat_history, ""

with gr.Blocks(theme=gr.themes.Soft(), title="Star Wars RAG Chatbot") as demo:
    gr.Markdown("""
    # 🌌 Star Wars RAG Chatbot
    Ask any question about the Star Wars universe and get answers from our knowledge base!
    """)

    chatbot = gr.Chatbot(label="Star Wars Expert", height=400, type="messages")
    txt = gr.Textbox(placeholder="Ask me anything about Star Wars...", container=False)
    submit_btn = gr.Button("Send")

    txt.submit(ask_question, inputs=[txt], outputs=[chatbot, txt])
    submit_btn.click(ask_question, inputs=[txt], outputs=[chatbot, txt])

demo.launch(debug=True)



/tmp/ipython-input-3100804124.py:42: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3100804124.py:42: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b8827e35be31ce1a68.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2127, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1904, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/components/chatbot.py", line 632, in postproce

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b8827e35be31ce1a68.gradio.live
